In [442]:
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer

In [443]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

In [444]:
#load data
def read_file(filename):
    with open(filename,'r') as f:
        data = f.read()
        data = json.loads(data)
    f.close()

    data_set = []
    data_label = []
    for i,t in data.items():
        data_set.append(t['text'])
        data_label.append(t['label'])
    
    return data_set, data_label

train_set, train_label = read_file('train_total_balance_new3.json')
dev_set, dev_label = read_file('dev.json')

with open('test-unlabelled.json','r') as f:
    test = f.read()
    test = json.loads(test)
    
test_set = []
for i,t in test.items():
    test_set.append(t['text'])

In [445]:
# remove url 
processed_train_set = [re.sub(r'http\S+', '', s) for s in train_set]
processed_dev_set = [re.sub(r'http\S+', '', s) for s in dev_set]
my_stopwords=stopwords.words('english')

# retain pronuons in stop words
remove_stopwords = ['i','they','he','she','we']
for w in remove_stopwords:
    my_stopwords.remove(w)

In [446]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+|\w+-\w+')

def my_tokenize(s):
    tokens = tokenizer.tokenize(s)
    tokens = [w.lower() for w in tokens]
    rem_stop_words  = [w for w in tokens if not w in my_stopwords]
    return " ".join(rem_stop_words)

# vectorize
vectorizer = CountVectorizer(min_df=1)
processed_train_set = [my_tokenize(s) for s in processed_train_set]
x_train = vectorizer.fit_transform(processed_train_set)
x_train

<2716x43972 sparse matrix of type '<class 'numpy.int64'>'
	with 726503 stored elements in Compressed Sparse Row format>

In [448]:
# generate tfidf features
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train)
x_train_tfidf.shape

(2716, 43972)

In [449]:
vectorizer_dev = CountVectorizer(min_df=1,vocabulary=vectorizer.vocabulary_)
processed_dev_set = [my_tokenize(s) for s in processed_dev_set]
x_dev = vectorizer_dev.fit_transform(processed_dev_set)
x_dev_tfidf = tfidf_transformer.fit_transform(x_dev)
print(x_dev_tfidf.shape)

(100, 43972)


In [450]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from tqdm import tqdm

# grid search on svm
grid_linear = {'penalty':['l2'],'C':[1,10,100,1000],'max_iter':[1000]}
clf = svm.LinearSVC(random_state=90042, tol=1e-4)

for z in tqdm(ParameterGrid(grid_linear)):
    clf.set_params(**z)
    clf.fit(x_train_tfidf, train_label)
    result = clf.predict(x_dev_tfidf)
    print(result)
#     unrelated = [21, 25, 28, 30, 33, 41, 46, 51, 55, 56, 58, 83, 87, 93, 95, 97]
#     for i in unrelated:
#         result[i] = 0
    p, r, f, _ = precision_recall_fscore_support(dev_label, result, pos_label=1, average="binary")
    print("===============================")
    print("pre:",p," recall:",r," f1:",f)
    print(z)



  0%|          | 0/1 [00:00<?, ?it/s]/Users/lizhengyang/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


100%|██████████| 1/1 [00:02<00:00,  2.01s/it]

[1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1
 1 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 1 1
 0 0 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 1 0 0 0 0 1]
pre: 0.875  recall: 0.84  f1: 0.8571428571428572
{'C': 100, 'max_iter': 1000, 'penalty': 'l2'}


In [451]:
# C = [0.1,1,10,100,1000,10000]
# gamma = ['scale',1e-4,1e-3,1e-2,1e-1,1,10]
C = [100]
gamma = [0.1]
grid_svc = {'C':C,'gamma':gamma}
clf1 = svm.SVC()
f1_lists = []
rrrr = []
for z in tqdm(ParameterGrid(grid_svc)):
    clf1.set_params(**z)
    clf1.fit(x_train_tfidf, train_label)
    result1 = clf1.predict(x_dev_tfidf)
    p, r, f, _ = precision_recall_fscore_support(dev_label, result1, pos_label=1, average="binary")
    f1_lists.append(f)
    print("===============================")
    print("pre:",p," recall:",r," f1:",f)
    print(z)



  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:10<00:00, 10.09s/it]

pre: 0.875  recall: 0.84  f1: 0.8571428571428572
{'C': 100, 'gamma': 0.1}


### Results
----------
Linear SVC
* pre: 0.8260869565217391  recall: 0.76  f1: 0.7916666666666667
{'C': 1, 'max_iter': 1000, 'penalty': 'l2', 'random_state': 0}    
----------
SVC
* pre: 0.918918918918919  recall: 0.68  f1: 0.7816091954022989
{'C': 0.1, 'gamma': 'scale', 'max_iter': 1000, 'random_state': 90042}
* pre: 0.8085106382978723  recall: 0.76  f1: 0.7835051546391754
{'C': 1, 'gamma': 'scale', 'max_iter': 1000, 'random_state': 90042}
* pre: 0.8260869565217391  recall: 0.76  f1: 0.7916666666666667
{'C': 5, 'gamma': 0.1, 'max_iter': 1000, 'random_state': 90042}
* pre: 0.8125  recall: 0.78  f1: 0.7959183673469388
{'C': 100, 'gamma': 0.1, 'max_iter': 1000, 'random_state': 90042}

* pre: 0.8125  recall: 0.78  f1: 0.7959183673469388
{'C': 1, 'gamma': 'scale'}   ---- max_df=0.95 min_df=5 

------------------
#### balance new
-------
Linear
pre: 0.7884615384615384  recall: 0.82  f1: 0.803921568627451
{'C': 1, 'max_iter': 1000, 'penalty': 'l2'}
pre: 0.7818181818181819  recall: 0.86  f1: 0.819047619047619
{'C': 100, 'max_iter': 1000, 'penalty': 'l2'}     

-------
SVC
pre: 0.7818181818181819  recall: 0.86  f1: 0.819047619047619
{'C': 100, 'gamma': 0.01}

pre: 0.8367346938775511  recall: 0.82  f1: 0.8282828282828283
{'C': 10, 'gamma': 0.1} new2

pre: 0.8431372549019608  recall: 0.86  f1: 0.8514851485148515
{'C': 100, 'gamma': 0.1} new1
pre: 0.8431372549019608  recall: 0.86  f1: 0.8514851485148515
{'C': 1000, 'gamma': 0.1}new1

In [454]:
# predict on test and give an output
processed_test_set = [re.sub(r'http\S+', '', s) for s in test_set]
vectorizer_test = CountVectorizer(min_df=1,vocabulary=vectorizer.vocabulary_)
processed_test_set = [my_tokenize(s) for s in processed_test_set]
x_test = vectorizer_test.fit_transform(processed_test_set)
# x_dev.shape
x_test_tfidf = tfidf_transformer.fit_transform(x_test)
print(x_test_tfidf.shape)

(1410, 43972)


In [456]:
result_test = clf.predict(x_test_tfidf)
print(result_test)
u_r = [3, 5, 6, 8, 13, 17, 20, 22, 23, 24, 28, 30, 32, 35, 36, 39, 45, 46, 47, 49, 50, 53, 54, 55, 60, 62, 64, 65, 66, 68, 69, 73, 74, 75, 78, 79, 80, 81, 82, 88, 89, 91, 93, 94, 95, 96, 97, 98, 100, 101, 102, 103, 106, 111, 114, 119, 120, 121, 122, 123, 131, 137, 138, 140, 141, 144, 145, 147, 149, 151, 152, 154, 159, 161, 163, 165, 167, 168, 170, 172, 175, 176, 178, 184, 187, 191, 193, 194, 196, 199, 200, 202, 204, 206, 209, 210, 211, 213, 216, 218, 220, 221, 225, 227, 228, 229, 230, 232, 235, 238, 239, 240, 241, 243, 245, 246, 250, 251, 252, 254, 258, 259, 262, 263, 264, 268, 271, 272, 276, 278, 279, 280, 284, 286, 288, 292, 297, 298, 299, 300, 301, 305, 310, 312, 313, 314, 315, 316, 317, 321, 322, 323, 325, 326, 327, 328, 331, 332, 333, 334, 335, 336, 338, 339, 343, 346, 349, 350, 351, 353, 354, 355, 360, 362, 365, 367, 369, 371, 373, 374, 375, 376, 378, 382, 384, 385, 393, 394, 395, 396, 398, 405, 412, 413, 416, 418, 420, 422, 423, 424, 425, 428, 429, 430, 431, 433, 435, 437, 438, 439, 440, 441, 442, 443, 444, 446, 448, 453, 454, 457, 459, 460, 461, 462, 466, 467, 468, 470, 471, 473, 474, 481, 482, 486, 489, 490, 493, 497, 500, 503, 504, 505, 507, 512, 513, 514, 515, 519, 522, 523, 528, 529, 530, 531, 532, 533, 534, 536, 537, 541, 548, 549, 551, 559, 562, 563, 565, 573, 574, 577, 585, 587, 588, 590, 597, 599, 600, 603, 606, 607, 609, 611, 613, 614, 618, 619, 621, 628, 635, 641, 646, 647, 649, 651, 652, 654, 656, 661, 662, 668, 669, 671, 674, 675, 676, 679, 680, 683, 685, 686, 689, 690, 694, 696, 697, 698, 699, 701, 702, 704, 705, 709, 710, 711, 712, 716, 718, 721, 725, 727, 728, 732, 733, 734, 736, 737, 739, 741, 744, 746, 747, 748, 751, 753, 754, 756, 757, 759, 761, 762, 766, 767, 774, 787, 788, 789, 790, 793, 794, 795, 796, 797, 803, 804, 812, 814, 818, 821, 823, 827, 829, 830, 832, 834, 836, 842, 843, 844, 847, 848, 849, 851, 853, 854, 855, 856, 858, 860, 861, 863, 864, 865, 866, 870, 871, 872, 878, 880, 883, 884, 889, 892, 893, 894, 898, 899, 901, 907, 909, 910, 912, 914, 915, 917, 918, 920, 921, 922, 923, 924, 925, 926, 930, 938, 945, 947, 949, 951, 952, 953, 955, 956, 957, 960, 962, 964, 966, 968, 969, 971, 972, 973, 975, 978, 980, 985, 986, 988, 989, 992, 997, 999, 1000, 1004, 1006, 1007, 1009, 1010, 1011, 1014, 1018, 1019, 1021, 1022, 1023, 1024, 1026, 1030, 1033, 1034, 1036, 1037, 1039, 1042, 1043, 1044, 1045, 1048, 1051, 1053, 1054, 1055, 1056, 1057, 1058, 1062, 1065, 1070, 1072, 1073, 1074, 1075, 1081, 1083, 1084, 1085, 1089, 1090, 1091, 1094, 1097, 1098, 1099, 1103, 1104, 1106, 1113, 1115, 1116, 1117, 1118, 1119, 1120, 1124, 1125, 1132, 1136, 1137, 1138, 1139, 1142, 1143, 1148, 1151, 1152, 1154, 1156, 1157, 1160, 1161, 1165, 1167, 1168, 1170, 1172, 1175, 1176, 1178, 1179, 1181, 1182, 1184, 1186, 1190, 1192, 1193, 1198, 1199, 1201, 1202, 1203, 1205, 1216, 1218, 1221, 1222, 1224, 1225, 1228, 1230, 1236, 1237, 1240, 1241, 1242, 1244, 1245, 1247, 1256, 1258, 1260, 1261, 1262, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1278, 1279, 1280, 1282, 1283, 1284, 1285, 1286, 1288, 1292, 1299, 1301, 1302, 1303, 1304, 1307, 1313, 1315, 1316, 1317, 1318, 1320, 1321, 1325, 1326, 1328, 1329, 1330, 1331, 1332, 1335, 1338, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1352, 1355, 1356, 1358, 1359, 1361, 1363, 1365, 1366, 1368, 1370, 1373, 1375, 1378, 1379, 1381, 1382, 1384, 1390, 1397, 1398, 1403, 1406]
# u_r = [3, 5, 6, 13, 20, 13, 14, 16, 17, 20, 21, 22, 24, 26, 29, 32, 33, 35, 36, 39, 40, 41, 43, 45, 46, 47, 48, 49, 52, 54, 55, 56, 64, 66, 67, 68, 69, 73, 74, 78, 82, 85, 87, 88, 90, 93, 94, 95, 98, 100, 101, 102, 103, 106, 107, 111, 114, 117, 118, 119, 120, 121, 122, 123, 124, 125, 131, 133, 137, 138, 139, 140, 141, 142, 144, 146, 151, 152, 157, 159, 161, 162, 163, 165, 167, 169, 170, 172, 176, 177, 179, 182, 184, 191, 193, 195, 199, 207, 209, 211, 213, 215, 216, 218, 220, 222, 223, 225, 228, 229, 230, 232, 238, 239, 241, 242, 243, 245, 246, 249, 252, 254, 255, 257, 259, 261, 262, 263, 264, 271, 272, 273, 274, 278, 279, 286, 288, 294, 297, 299, 302, 306, 307, 308, 309, 310, 312, 316, 317, 321, 322, 325, 327, 329, 331, 332, 334, 335, 343, 344, 346, 349, 350, 354, 355, 359, 360, 371, 373, 374, 376, 378, 385, 389, 392, 394, 395, 396, 399, 405, 409, 411, 412, 414, 416, 418, 419, 422, 424, 426, 428, 429, 430, 431, 433, 435, 437, 440, 441, 442, 445, 446, 448, 450, 452, 453, 454, 456, 457, 462, 466, 467, 470, 471, 473, 474, 476, 480, 482, 488, 489, 490, 493, 495, 497, 501, 502, 503, 504, 505, 507, 511, 512, 514, 519, 522, 523, 527, 528, 529, 530, 532, 534, 541, 542, 546, 547, 548, 554, 562, 563, 567, 568, 573, 574, 576, 577, 578, 581, 582, 584, 585, 586, 587, 588, 589, 597, 598, 599, 600, 601, 602, 603, 609, 614, 615, 618, 620, 621, 622, 623, 624, 628, 635, 638, 647, 649, 651, 652, 654, 665, 668, 671, 673, 675, 677, 680, 681, 683, 684, 686, 689, 692, 696, 697, 699, 701, 702, 704, 705, 708, 710, 711, 712, 714, 715, 716, 717, 723, 725, 732, 733, 734, 736, 739, 740, 741, 744, 746, 747, 748, 751, 753, 755, 756, 757, 761, 764, 767, 771, 774, 775, 782, 791, 794, 795, 796, 797, 801, 804, 810, 811, 812, 814, 815, 817, 818, 819, 823, 827, 829, 830, 832, 835, 836, 837, 843, 844, 849, 850, 851, 853, 854, 855, 856, 861, 863, 864, 865, 867, 868, 870, 872, 880, 883, 884, 892, 893, 894, 899, 900, 901, 902, 906, 907, 908, 909, 910, 911, 912, 914, 915, 916, 917, 918, 919, 920, 924, 926, 928, 929, 930, 931, 934, 941, 943, 945, 947, 951, 953, 955, 956, 957, 960, 962, 967, 971, 973, 977, 978, 980, 982, 984, 985, 986, 990, 992, 994, 995, 999, 1000, 1007, 1009, 1010, 1011, 1014, 1018, 1019, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1029, 1032, 1033, 1037, 1038, 1039, 1044, 1045, 1049, 1051, 1053, 1054, 1055, 1056, 1057, 1058, 1060, 1062, 1063, 1069, 1072, 1074, 1075, 1079, 1081, 1083, 1084, 1085, 1089, 1091, 1094, 1095, 1097, 1098, 1103, 1106, 1112, 1113, 1114, 1116, 1119, 1120, 1121, 1123, 1129, 1132, 1135, 1136, 1138, 1139, 1142, 1144, 1148, 1151, 1152, 1154, 1156, 1157, 1159, 1164, 1166, 1168, 1172, 1175, 1176, 1178, 1179, 1180, 1181, 1185, 1186, 1191, 1193, 1194, 1195, 1198, 1199, 1200, 1203, 1216, 1218, 1219, 1221, 1222, 1224, 1228, 1229, 1230, 1232, 1233, 1236, 1237, 1239, 1240, 1241, 1242, 1247, 1249, 1251, 1254, 1255, 1256, 1258, 1259, 1261, 1263, 1264, 1265, 1267, 1269, 1270, 1271, 1273, 1276, 1278, 1279, 1280, 1282, 1283, 1284, 1287, 1288, 1289, 1291, 1295, 1300, 1301, 1302, 1303, 1304, 1307, 1313, 1315, 1316, 1318, 1320, 1321, 1323, 1326, 1328, 1330, 1332, 1333, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1355, 1360, 1361, 1363, 1366, 1368, 1370, 1371, 1375, 1378, 1382, 1383, 1384, 1387, 1390, 1393, 1397, 1398, 1403, 1404, 1406]
for i in u_r:
    result_test[i] = 0
result_dict_test = {}
for i in range(len(result_test)):
    key = "test-{}".format(i)
    result_dict_test[key] = {"label":int(result_test[i])}

with open("test-output.json","w") as f:
    json.dump(result_dict_test,f)
    print("finish")

[0 0 0 ... 1 1 0]
finish
